**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings_count = ratings.groupby('userId').count().reset_index()
ratings_count = ratings_count[['userId', 'rating']]
count_more_than_100 = ratings_count[ratings_count['rating'] > 100]

ratings_timestamp = ratings[['userId', 'timestamp']]
rating_max = ratings_timestamp.groupby('userId').max().reset_index()
rating_min = ratings_timestamp.groupby('userId').min().reset_index()
rating_max.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
rating_min.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)

avg_timestamp = count_more_than_100.merge(rating_max, how='inner').merge(rating_min, how='inner')

def average(row):
    avg = (row['timestamp_max'] - row['timestamp_min'])
    return avg


avg_timestamp['avg'] = avg_timestamp.apply(average, axis=1)
print(avg_timestamp.head())

   userId  rating  timestamp_max  timestamp_min        avg
0       4     204      949982274      949778714     203560
1       8     116     1154474527     1154389340      85187
2      15    1700     1469330735      997937239  471393496
3      17     363     1127476640     1127468587       8053
4      19     423      855195373      855190091       5282


**Задание 2**

Задание 2
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [3]:
urls = pd.read_csv('URLs.txt')
print(urls.url[:8])

0                                              /world/
1                                             /latest/
2                                        /?updated=top
3    /politics/36188461-s-marta-zhizn-rossiyan-susc...
4    /world/36007585-tramp-pridumal-kak-reshit-ukra...
5    /science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6    /video/36001498-poyavilis-pervye-podrobnosti-g...
7    /world/36007585-tramp-pridumal-kak-reshit-ukra...
Name: url, dtype: object


In [4]:
url_split = pd.DataFrame([i for i in urls.url if pd.Series(i).str.contains('\/\d{8}-', regex=True)[0]],columns=['URL'])
print(url_split.head(5))

                                                 URL
0  /politics/36188461-s-marta-zhizn-rossiyan-susc...
1  /world/36007585-tramp-pridumal-kak-reshit-ukra...
2  /science/36157853-nasa-sobiraet-ekstrennuyu-pr...
3  /video/36001498-poyavilis-pervye-podrobnosti-g...
4  /world/36007585-tramp-pridumal-kak-reshit-ukra...


**Задание 3**

В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [12]:
from pymystem3 import Mystem
m = Mystem()

data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

def get_words(row):
    lemmas = ' '.join(m.lemmatize(row['keyword']))
    return lemmas.rstrip()

data['lemmas'] = data.apply(get_words, axis=1)
data.head()

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль
1,доллары в рубли,114173,доллар в рубль
2,100 долларов в рублях,97534,100 доллар в рубль
3,курс рубля,53546,курс рубль


In [10]:
import sys
!{sys.executable} -m pip install pymystem3